In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vaccine-stuff/US_vaccination_search_insights.csv
/kaggle/input/vaccine-stuff/US_l2_vaccination_trending_searches.csv
/kaggle/input/vaccine/US_l1_vaccination_trending_searches.csv


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset into a pandas DataFrame
df = pd.read_csv('/kaggle/input/vaccine/US_l1_vaccination_trending_searches.csv')

newdf = df.drop(columns=["country_region", "country_region_code", "sub_region_1_code", "sub_region_2", "sub_region_2_code", "sub_region_3", "sub_region_3_code", "place_id"])

# Filter the dataset to only include data for Massachusetts
ma_df = newdf[newdf['sub_region_1'] == 'Massachusetts']

# Print the first few rows of the DataFrame
# print(ma_df.head())

In [3]:
# Split the dataset into features and labels
X = ma_df.drop(['num_members'], axis=1)
y = ma_df['num_members']

# Convert the date column to datetime and extract the ordinal value
X['date'] = pd.to_datetime(X['date'])
X['date'] = X['date'].apply(lambda x: x.toordinal())

In [4]:
# Encode categorical variables
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
from tensorflow.keras.regularizers import l2

# Define the model architecture with L2 regularization
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1)
])
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
146/146 [==============================] - 3s 13ms/step - loss: 32056.5039 - val_loss: 29717.3359
Epoch 2/50
146/146 [==============================] - 1s 10ms/step - loss: 24839.4473 - val_loss: 27360.7520
Epoch 3/50
146/146 [==============================] - 2s 10ms/step - loss: 18055.8926 - val_loss: 23972.5312
Epoch 4/50
146/146 [==============================] - 2s 11ms/step - loss: 10897.0986 - val_loss: 20886.2891
Epoch 5/50
146/146 [==============================] - 2s 11ms/step - loss: 6508.6270 - val_loss: 19503.1680
Epoch 6/50
146/146 [==============================] - 1s 10ms/step - loss: 3965.4487 - val_loss: 18032.1309
Epoch 7/50
146/146 [==============================] - 2s 10ms/step - loss: 2766.5112 - val_loss: 16861.1387
Epoch 8/50
146/146 [==============================] - 2s 10ms/step - loss: 2171.9055 - val_loss: 16465.8301
Epoch 9/50
146/146 [==============================] - 2s 11ms/step - loss: 1763.8751 - val_loss: 16270.7100
Epoch 10/50
146/146 [===

In [7]:
# Generate predictions on the test set
y_pred = model.predict(X_test)

# Print the first 10 predictions and actual values
print('Predictions:', y_pred[:10])
print('Actual values:', y_test[:10])

46/46 [==============================] - 0s 4ms/step
Predictions: [[155.51233]
 [539.4351 ]
 [468.2014 ]
 [145.45341]
 [335.32574]
 [184.6559 ]
 [291.8441 ]
 [134.01047]
 [153.2817 ]
 [247.10983]]
Actual values: 194502      1
45851     259
46579     266
194859      1
194961     24
129962     58
194105      5
194838      2
130144      5
128983    151
Name: num_members, dtype: int64


In [8]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)

print('Test loss:', loss)

46/46 [==============================] - 0s 3ms/step - loss: 71027.5391
Test loss: 71027.5390625


In [9]:
# Make predictions on the test set
predicted_vaccine_rates = model.predict(X_test)

print(predicted_vaccine_rates)

46/46 [==============================] - 0s 3ms/step
[[155.51233]
 [539.4351 ]
 [468.2014 ]
 ...
 [ 17.63497]
 [313.0178 ]
 [167.3139 ]]


In [12]:
num_simulations = 1000

# Initialize array to store simulation results
sim_results = np.zeros(num_simulations)

# Run simulations
for i in range(num_simulations):
    # Sample from distribution of predicted vaccine uptake rates
    sim_vaccine_rates = np.random.normal(predicted_vaccine_rates.mean(), predicted_vaccine_rates.std(), size=len(X_test))
    
    # Calculate simulated number of people who will get the vaccine
    sim_vaccine_counts = sim_vaccine_rates * 6.985
    
    # Sum total number of people who will get the vaccine across all regions
    sim_results[i] = sim_vaccine_counts.sum()

# Calculate mean and standard deviation of simulation results
sim_mean = sim_results.mean()/1
sim_std = sim_results.std()/1

# Print summary statistics
print("Simulation results:")
print(f"Mean: {sim_mean}")
print(f"Standard deviation: {sim_std}")


Simulation results:
Mean: 4173235.4508284917
Standard deviation: 54873.24505207033


In [15]:
print(sum(y_pred))

[597916.6]
